In [1]:
import pandas as pd
import numpy as np
import nibabel as nib
import importlib
import tTsTGrpUtils as tsutil
import pickle
importlib.reload(tsutil)

<module 'tTsTGrpUtils' from '/host/verges/tank/data/daniel/3T7T/z/code/analyses/tTsTGrpUtils.py'>

In [2]:
# LOAD DATA
demographics = {
    "pth" : "/host/verges/tank/data/daniel/3T7T/z/data/pt/demo_27Aug2025.csv",
    # column names:
    'nStudies': True, # whether multiple studies are included
    "ID_7T" : "PNI_ID", 
    "ID_3T" : "MICS_ID",
    "SES" : "SES",
    "date": "Date",
    "age": "age",
    "sex": "sex",
    "grp" : "grp_detailed" # col name for participant grouping variable of interest
}


pth = "/host/verges/tank/data/daniel/3T7T/z/outputs/"

file_maps = "01_maps_09Sep2025-162453.pkl" # smoothed maps?
file_stats = "02_stats_10Sep2025-091747.pkl" # z, w scores for all participnats
file_stats_grp = "03_stats_grp_10Sep2025-163811.pkl" #  subsetted by group with ic
file_dStats = "04_dStats_grp_10Sep2025-180606.pkl" # d between grp and controls within study
file_comps = "05_comps_11Sep2025-170446.pkl" # d differences between 7T and 3T 

with open(pth + file_maps, "rb") as f:
    maps = pickle.load(f)

with open(pth + file_stats, "rb") as f:
    stats = pickle.load(f)

with open(pth + file_stats_grp, "rb") as f:
    stats_grp = pickle.load(f)

with open(pth + file_dStats, "rb") as f:
    dStats = pickle.load(f)

with open(pth + file_comps, "rb") as f:
    comps = pickle.load(f)

print("========= maps " + "="*100)
tsutil.print_dict(maps, df_print = False)
print("========= stats " + "="*100)
tsutil.print_dict(stats, df_print = False)
print("========= stats_grp " + "="*100)
tsutil.print_dict(stats_grp, df_print = False)
print("========= dStats" + "="*100)
tsutil.print_dict(dStats, df_print = False)
print("========= comps "+ "="*100)
tsutil.print_dict(comps, df_print = False)

========= maps ====================================================================================================

 Dict list length (24 items)

[0]
	Keys: ['study', 'region', 'surf', 'label', 'feature', 'smth', 'df_demo', 'df_maps']
	study: MICs
	region: cortex
	surf: fsLR-5k
	label: thickness
	feature: thickness
	smth: 5
	df_demo: <DataFrame shape=(54, 71)>
	df_maps: <DataFrame shape=(54, 9686)>

[1]
	Keys: ['study', 'region', 'surf', 'label', 'feature', 'smth', 'df_demo', 'df_maps']
	study: PNI
	region: cortex
	surf: fsLR-5k
	label: thickness
	feature: thickness
	smth: 5
	df_demo: <DataFrame shape=(54, 71)>
	df_maps: <DataFrame shape=(54, 9686)>

[2]
	Keys: ['study', 'region', 'surf', 'label', 'feature', 'smth', 'df_demo', 'df_maps']
	study: MICs
	region: cortex
	surf: fsLR-5k
	label: thickness
	feature: thickness
	smth: 10
	df_demo: <DataFrame shape=(54, 71)>
	df_maps: <DataFrame shape=(54, 9686)>

[3]
	Keys: ['study', 'region', 'surf', 'label', 'feature', 'smth', 'df_demo', 'df_

In [31]:
maps[1]['df_maps'].index

RangeIndex(start=0, stop=54, step=1)

In [4]:
# PLOT SMTH MAPS
import time
importlib.reload(tsutil)

df_name = 'df_maps'
metric = '1' # indices in maps is participant number
ipsiTo = None

save_pth = "/host/verges/tank/data/daniel/3T7T/z/outputs/figs_debug_12Sep2025/01_maps"

date = time.strftime("%d%b%Y")
for i, item in enumerate(maps):
    tsutil.printItemMetadata(item, i)
    # print ID-SES for the case
    
    col_ID = tsutil.get_IDCol(item['study'], demographics)
    df = item[df_name]
    id = df.loc[int(metric), col_ID]
    ses = df.loc[int(metric), 'SES'] if 'SES' in df.columns else ''
    print(f"\t\tID: {id}, SES: {ses}")

    save_name = f"01_maps_sub-{item['study']}_{item['region']}_{item['feature']}_{item['surf']}_{item['label']}_{item['smth']}mm_{id}-{ses}_{date}"
    tsutil.itmToVisual(
                        item=item, 
                        df_name = df_name, 
                        metric = metric, 
                        ipsiTo = ipsiTo,
                        save_pth = save_pth,
                        save_name = save_name
                        ) # need to change max values dynamically based on data range

	[MICs] - cortex: thickness, fsLR-5k, thickness, 5mm (idx 0)
		ID: HC052, SES: 01
[showBrains] WARNING: MICAPIPE environment variable not set. Using hard-coded path /data_/mica1/01_programs/micapipe-v0.2.0
[showBrains] Plot saved to 01_maps_sub-MICs_cortex_thickness_fsLR-5k_thickness_5mm_HC052-01_12Sep2025_fsLR-5k_12Sep2025-1023.png
	[PNI] - cortex: thickness, fsLR-5k, thickness, 5mm (idx 1)
		ID: PNC006, SES: 01
[showBrains] WARNING: MICAPIPE environment variable not set. Using hard-coded path /data_/mica1/01_programs/micapipe-v0.2.0
[showBrains] Plot saved to 01_maps_sub-PNI_cortex_thickness_fsLR-5k_thickness_5mm_PNC006-01_12Sep2025_fsLR-5k_12Sep2025-1023.png
	[MICs] - cortex: thickness, fsLR-5k, thickness, 10mm (idx 2)
		ID: HC052, SES: 01
[showBrains] WARNING: MICAPIPE environment variable not set. Using hard-coded path /data_/mica1/01_programs/micapipe-v0.2.0
[showBrains] Plot saved to 01_maps_sub-MICs_cortex_thickness_fsLR-5k_thickness_10mm_HC052-01_12Sep2025_fsLR-5k_12Sep2025-10

In [7]:
stats[0]['df_z']

,0_L,1_L,2_L,3_L,4_L,5_L,6_L,7_L,8_L,9_L,...,4832_R,4833_R,4834_R,4835_R,4836_R,4837_R,4838_R,4839_R,4840_R,4841_R
ID_SES,,,,,,,,,,,,,,,,,,,,,
HC013_01,0.287763,1.444836,0.787503,2.323558,-0.060812,1.389242,0.493706,-0.015198,0.358242,1.369109,...,-1.061103,-1.816318,-1.728891,0.179470,-1.430046,-1.603440,-0.823384,-1.538052,-1.232437,-1.065884
HC052_01,-0.066726,1.459822,-0.377864,-0.214950,1.770984,-1.128763,0.818855,-2.921021,0.421983,0.480379,...,1.288755,0.151251,-0.797864,-0.188728,0.524385,0.032365,-0.509466,0.627244,0.170960,0.241152
HC062_01,0.419042,-0.173270,-2.246080,-0.373918,-1.424922,0.509089,-0.632110,-1.430740,0.194752,0.226506,...,-0.079933,0.173432,-0.343114,-0.492316,0.212542,-0.237270,-0.835415,0.012319,-1.415112,-1.459505
HC069_01,1.300512,1.057804,-0.247607,0.370192,2.339656,0.404922,-1.767557,0.902987,-0.549907,1.271670,...,0.337861,0.708528,1.356238,1.268822,0.580267,0.879577,1.508768,0.538142,1.075602,0.974267
HC076_01,-2.001283,0.296967,0.029177,0.066855,-0.197014,0.179935,0.064223,0.008078,-1.263010,0.294257,...,-1.179186,0.004238,0.294816,-0.470786,0.212553,0.378582,0.071527,0.065190,0.121995,0.270705
HC081_01,-0.065225,1.187677,-0.301737,0.831690,0.312722,0.538606,-0.101014,0.897421,1.062203,1.065414,...,0.421550,1.198716,1.618589,-0.151218,0.881692,1.733069,1.124984,1.908215,1.561525,1.619308
HC082_01,-0.866475,-0.333267,-0.375852,-0.543542,0.444053,1.271827,1.510568,0.348042,0.270203,0.386463,...,0.044670,0.702095,0.285588,-1.391939,1.006625,0.520800,-1.252081,0.469464,-0.988126,-0.902906
HC083_01,-0.882016,-0.358858,0.418292,-0.674603,-1.262932,0.890743,-1.086319,0.625099,-0.686534,-1.292055,...,0.124762,-1.104334,-0.409255,0.814920,-0.263667,-0.245271,0.762156,-0.138296,0.357556,-0.822509
HC088_01,2.412404,0.520279,1.927005,1.740275,0.635541,0.806861,0.344336,0.462507,0.710823,1.985596,...,1.156264,0.693316,1.289963,1.086330,0.631464,0.733593,1.150835,0.894567,1.310974,1.698917


In [6]:
# PLOT STATS MAPS
import time

df_stats = ['df_z', 'df_w']
df_name = ['z', 'w']

ipsiTo = None

save_pth = "/host/verges/tank/data/daniel/3T7T/z/outputs/figs_debug_12Sep2025/02_stats"

date = time.strftime("%d%b%Y")

for i, item in enumerate(stats):
    tsutil.printItemMetadata(item, i)
    # print ID-SES for the case
    
    for name, df in zip(df_name, df_stats):

        if item.get(df, None) is None:
            print(f"Item None or not in list Skipping.")
            continue
        # choose a random row
        idx = np.random.choice(item[df].index)
        metric = idx    
        
        print(f"Stat: {name}. Random participant {metric}")

        save_name = f"02_stats_sub-{item['study']}_{item['region']}_{item['feature']}_{item['surf']}_{item['label']}_{item['smth']}mm_{metric}_{name}_{date}"
        tsutil.itmToVisual(
                            item=item, 
                            df_name = df, 
                            metric = metric, 
                            ipsiTo = ipsiTo,
                            save_pth = save_pth,
                            save_name = save_name,
                            min_val= -2, max_val = 2,
                            title=f"Stat: {name}, Sub: {metric}"
                            )

	[MICs] - cortex: thickness, fsLR-5k, thickness, 5mm (idx 0)
Stat: z. Random participant HC135_01
[showBrains] WARNING: MICAPIPE environment variable not set. Using hard-coded path /data_/mica1/01_programs/micapipe-v0.2.0
[showBrains] Plot saved to 02_stats_sub-MICs_cortex_thickness_fsLR-5k_thickness_5mm_HC135_01_z_12Sep2025_fsLR-5k_12Sep2025-1114.png
Stat: w. Random participant PX050_01
[showBrains] WARNING: MICAPIPE environment variable not set. Using hard-coded path /data_/mica1/01_programs/micapipe-v0.2.0
[showBrains] Plot saved to 02_stats_sub-MICs_cortex_thickness_fsLR-5k_thickness_5mm_PX050_01_w_12Sep2025_fsLR-5k_12Sep2025-1114.png
	[PNI] - cortex: thickness, fsLR-5k, thickness, 5mm (idx 1)
Stat: z. Random participant PNC011_01
[showBrains] WARNING: MICAPIPE environment variable not set. Using hard-coded path /data_/mica1/01_programs/micapipe-v0.2.0
[showBrains] Plot saved to 02_stats_sub-PNI_cortex_thickness_fsLR-5k_thickness_5mm_PNC011_01_z_12Sep2025_fsLR-5k_12Sep2025-1114.png

In [11]:
stats_grp[0].keys()

dict_keys(['study', 'region', 'surf', 'label', 'feature', 'smth', 'df_demo', 'df_maps', 'ctrl_IDs', 'df_z', 'df_w', 'df_w_models', 'TLE_IDs_R', 'TLE_IDs_L', 'df_z_TLE_R', 'df_z_TLE_L', 'df_z_TLE_ic', 'df_z_ctrl', 'df_w_TLE_R', 'df_w_TLE_L', 'df_w_TLE_ic', 'df_w_ctrl'])

In [ ]:
# PLOT STATS_GRP MAPS
import time

df_stats = ['df_w_ctrl', 'df_z_ctrl','df_z_TLE_ic', 'df_w_TLE_ic', 'df_z_TLE_R', 'df_w_TLE_R', 'df_z_TLE_L', 'df_w_TLE_L']
df_name = ['w_ctrl', 'z_ctrl','z_TLE_ic', 'w_TLE_ic', 'z_TLE_R', 'w_TLE_R', 'z_TLE_L', 'w_TLE_L']
ipsiTo = None

save_pth = "/host/verges/tank/data/daniel/3T7T/z/outputs/figs_debug_12Sep2025/03_stats_grp"

date = time.strftime("%d%b%Y")

for i, item in enumerate(stats_grp):
    tsutil.printItemMetadata(item, i)
    # print ID-SES for the case
    for name, df in zip(df_name, df_stats):
        if item.get(df, None) is None:
            print(f"Item None or not in list Skipping.")
            continue

        # choose a random row
        idx = np.random.choice(item[df].index)
        metric = idx
        print(f"Stat: {name}. ID: {metric}")

        # set ipsiTo based on name containing 'ic'
        if 'ic' in name:
            ipsiTo = 'L'
        else:
            ipsiTo = None
            
        save_name = f"03_statsGrp_sub-{item['study']}_{item['region']}_{item['feature']}_{item['surf']}_{item['label']}_{item['smth']}mm_{metric}_{name}_{date}"
        tsutil.itmToVisual(
                            item=item, 
                            df_name = df, 
                            metric = metric, 
                            ipsiTo = ipsiTo,
                            save_pth = save_pth,
                            save_name = save_name,
                            min_val= -2, max_val = 2,
                            title=f"Stat: {name}, Sub: {metric}"
                            )

	[MICs] - cortex: thickness, fsLR-5k, thickness, 5mm (idx 0)
Stat: w_ctrl. Metric: HC088_01
[showBrains] WARNING: MICAPIPE environment variable not set. Using hard-coded path /data_/mica1/01_programs/micapipe-v0.2.0
[showBrains] Plot saved to 03_statsGrp_sub-MICs_cortex_thickness_fsLR-5k_thickness_5mm_HC088_01_w_ctrl_12Sep2025_fsLR-5k_12Sep2025-1120.png
Stat: z_ctrl. Metric: HC116_01
[showBrains] WARNING: MICAPIPE environment variable not set. Using hard-coded path /data_/mica1/01_programs/micapipe-v0.2.0
[showBrains] Plot saved to 03_statsGrp_sub-MICs_cortex_thickness_fsLR-5k_thickness_5mm_HC116_01_z_ctrl_12Sep2025_fsLR-5k_12Sep2025-1120.png
Stat: z_TLE_ic. Metric: PX204_01
[showBrains] WARNING: MICAPIPE environment variable not set. Using hard-coded path /data_/mica1/01_programs/micapipe-v0.2.0
[showBrains] Plot saved to 03_statsGrp_sub-MICs_cortex_thickness_fsLR-5k_thickness_5mm_PX204_01_z_TLE_ic_12Sep2025_fsLR-5k_12Sep2025-1120.png
Stat: w_TLE_ic. Metric: PX148_01
[showBrains] WARN

: 

In [22]:
dStats[1]['df_d'].index.tolist()

['m_z_ctrl',
 'std_z_ctrl',
 'm_df_z_TLE_R',
 'std_df_z_TLE_R',
 'd_df_z_TLE_R',
 'm_df_z_TLE_L',
 'std_df_z_TLE_L',
 'd_df_z_TLE_L',
 'm_w_ctrl',
 'std_w_ctrl',
 'm_df_w_TLE_R',
 'std_df_w_TLE_R',
 'd_df_w_TLE_R',
 'm_df_w_TLE_L',
 'std_df_w_TLE_L',
 'd_df_w_TLE_L']

In [38]:
importlib.reload(tsutil)

ModuleNotFoundError: spec not found for the module 'tTsTGrpUtils'

In [ ]:
# PLOT D MAPS
import time

'm_z_ctrl', 'std_z_ctrl', 'm_df_z_TLE_R', 'std_df_z_TLE_R', 'd_df_z_TLE_R', 'm_df_z_TLE_L', 
'std_df_z_TLE_L', 'd_df_z_TLE_L', 'm_w_ctrl', 'std_w_ctrl', 'm_df_w_TLE_R', 'std_df_w_TLE_R',
'd_df_w_TLE_R', 'm_df_w_TLE_L', 'std_df_w_TLE_L', 'd_df_w_TLE_L'


#df_stats = ['df_d', 'df_d_ic']
df_stats = ['df_d_ic']
metrics = ['m_z_ctrl', 'std_z_ctrl', 'm_df_z_TLE_R', 'std_df_z_TLE_R','d_df_z_TLE_R', 'm_df_z_TLE_L', 'std_df_z_TLE_L', 'd_df_z_TLE_L']
#'m_w_ctrl', 'std_w_ctrl', 'm_df_w_TLE_R', 'std_df_w_TLE_R','d_df_w_TLE_R', 'm_df_w_TLE_L', 'std_df_w_TLE_L', 'd_df_w_TLE_L'
ipsiTo = None

save_pth = "/host/verges/tank/data/daniel/3T7T/z/outputs/figs_debug_12Sep2025/04_dStats"

date = time.strftime("%d%b%Y")

for i, item in enumerate(dStats):
    tsutil.printItemMetadata(item, i)

    for df_stat in df_stats:
        print(f"Processing {df_stat}...")
        if item.get(df_stat, None) is None:
            print(f"Item None or not in list Skipping.")
            continue
        
        metrics = item[df_stat].index.tolist()
        #print(metrics)
        
        # set ipsiTo based on name containing 'ic'
        if 'ic' in df_stat:
            ipsiTo = 'L'
        else:
            ipsiTo = None
            
        for metric in metrics:
            print("-"*100)
            print(f"\t\t{metric}")

            if "std" in metric:
                min_val = 0
                max_val = 3
                #print(f"\t\tSetting min/max for std: {min_val}, {max_val}")
            else:
                min_val = -2
                max_val = 2

            save_name = f"03_statsGrp_sub-{item['study']}_{item['region']}_{item['feature']}_{item['label']}_{item['smth']}mm_{metric}_{df_stat}"
            tsutil.itmToVisual(
                                item=item, 
                                df_name = df_stat, 
                                metric = metric, 
                                ipsiTo = ipsiTo,
                                save_pth = save_pth,
                                save_name = save_name,
                                min_val= min_val, max_val = max_val,
                                title=None
                                )

	[MICs] - cortex: thickness, fsLR-5k, thickness, 5mm (idx 0)
Processing df_d_ic...
IC in name
Index(['0_ipsi', '1_ipsi', '2_ipsi', '3_ipsi', '4_ipsi', '5_ipsi', '6_ipsi',
       '7_ipsi', '8_ipsi', '9_ipsi',
       ...
       '4832_contra', '4833_contra', '4834_contra', '4835_contra',
       '4836_contra', '4837_contra', '4838_contra', '4839_contra',
       '4840_contra', '4841_contra'],
      dtype='object', length=9684)
----------------------------------------------------------------------------------------------------
		m_z_ctrl
[showBrains] WARNING: MICAPIPE environment variable not set. Using hard-coded path /data_/mica1/01_programs/micapipe-v0.2.0
[showBrains] Plot saved to 03_statsGrp_sub-MICs_cortex_thickness_thickness_5mm_m_z_ctrl_df_d_ic_fsLR-5k_12Sep2025-1206.png
----------------------------------------------------------------------------------------------------
		std_z_ctrl
[showBrains] WARNING: MICAPIPE environment variable not set. Using hard-coded path /data_/mica1/01_pr

: 

In [4]:
comps[1]['comps_df_d_ic'].index.tolist()

['d_df_z_TLE_ic_ipsiTo-L_Δd',
 'd_df_w_TLE_ic_ipsiTo-L_Δd',
 'd_df_z_TLE_ic_ipsiTo-L_Δd_by3T',
 'd_df_w_TLE_ic_ipsiTo-L_Δd_by3T',
 'd_df_z_TLE_ic_ipsiTo-L_Δd_by7T',
 'd_df_w_TLE_ic_ipsiTo-L_Δd_by7T',
 'm_z_ctrl_3T',
 'std_z_ctrl_3T',
 'm_df_z_TLE_ic_ipsiTo-L_3T',
 'std_df_z_TLE_ic_ipsiTo-L_3T',
 'd_df_z_TLE_ic_ipsiTo-L_3T',
 'm_w_ctrl_3T',
 'std_w_ctrl_3T',
 'm_df_w_TLE_ic_ipsiTo-L_3T',
 'std_df_w_TLE_ic_ipsiTo-L_3T',
 'd_df_w_TLE_ic_ipsiTo-L_3T',
 'm_z_ctrl_7T',
 'std_z_ctrl_7T',
 'm_df_z_TLE_ic_ipsiTo-L_7T',
 'std_df_z_TLE_ic_ipsiTo-L_7T',
 'd_df_z_TLE_ic_ipsiTo-L_7T',
 'm_w_ctrl_7T',
 'std_w_ctrl_7T',
 'm_df_w_TLE_ic_ipsiTo-L_7T',
 'std_df_w_TLE_ic_ipsiTo-L_7T',
 'd_df_w_TLE_ic_ipsiTo-L_7T']

In [ ]:
# PLOT COMPS
import time

#df_stats = ['df_d', 'df_d_ic']
df_stats = ['comps_df_d_ic', 'comps_df_d']
metrics_d = ['d_df_z_TLE_R_Δd', 'd_df_z_TLE_L_Δd', 'd_df_w_TLE_R_Δd', 'd_df_w_TLE_L_Δd', 'd_df_z_TLE_R_Δd_by3T', 'd_df_z_TLE_L_Δd_by3T', 'd_df_w_TLE_R_Δd_by3T', 'd_df_w_TLE_L_Δd_by3T', 'd_df_z_TLE_R_Δd_by7T', 'd_df_z_TLE_L_Δd_by7T', 'd_df_w_TLE_R_Δd_by7T', 'd_df_w_TLE_L_Δd_by7T']
metrics_ic = ['d_df_z_TLE_ic_ipsiTo-L_Δd', 'd_df_w_TLE_ic_ipsiTo-L_Δd', 'd_df_z_TLE_ic_ipsiTo-L_Δd_by3T', 'd_df_w_TLE_ic_ipsiTo-L_Δd_by3T', 'd_df_z_TLE_ic_ipsiTo-L_Δd_by7T', 'd_df_w_TLE_ic_ipsiTo-L_Δd_by7T']
df_metrics = [metrics_ic, metrics_d]
ipsiTo = None

save_pth = "/host/verges/tank/data/daniel/3T7T/z/outputs/figs_debug_12Sep2025/05_comps"

date = time.strftime("%d%b%Y")

for idx, item in enumerate(comps):
    tsutil.printItemMetadata(item)
    print(df_stats)
    
    for df_stat, metrics in zip(df_stats, df_metrics):
        print(f"Processing {df_stat}...")
        if item.get(df_stat, None) is None:
            print(f"Item None or not in list Skipping.")
            continue

        # set ipsiTo based on name containing 'ic'
        if 'ic' in df_stat:
            ipsiTo = 'L'
        else:
            ipsiTo = None
            
        for metric in metrics:
            print("-"*100)
            print(f"\t\t{metric}")
            continue
            min_val = -2
            max_val = 2

            save_name = f"05_comps_{item['region']}_{item['feature']}_{item['label']}_{item['smth']}mm_{metric}_{df_stat}"
            tsutil.itmToVisual(
                                item=item, 
                                df_name = df_stat, 
                                metric = metric, 
                                ipsiTo = ipsiTo,
                                save_pth = save_pth,
                                save_name = save_name,
                                min_val= min_val, max_val = max_val,
                                title=None
                                )

	[('MICs', 'PNI')] - cortex: thickness, fsLR-5k, thickness, 5mm
['comps_df_d_ic', 'comps_df_d']
Processing comps_df_d_ic...
----------------------------------------------------------------------------------------------------


NameError: name 'metric' is not defined

In [ ]:
# ridge plots for individual patients, import dictionary lists with raw maps
# import dictionary lists with raw maps


demographics = {
    "pth" : "/host/verges/tank/data/daniel/3T7T/z/data/pt/demo_27Aug2025.csv",
    # column names:
    'nStudies': True, # whether multiple studies are included
    "ID_7T" : "PNI_ID", 
    "ID_3T" : "MICS_ID",
    "SES" : "SES",
    "date": "Date",
    "age": "age",
    "sex": "sex",
    "grp" : "grp_detailed" # col name for participant grouping variable of interest
}
#tsutil.print_dict(dl, df_print=False)

In [ ]:
# ridge plots
for i in range(len(dl)):
    df_demo = dl[i]['df_demo']
    df_maps = dl[i]['df_maps']
    label = dl[i]['label']
    surface = dl[i]['surf']
    region = dl[i]['region']
    feature = dl[i]['feature']
    study = dl[i]['study']
    smth = dl[i]['smth']
    print(f"[{study}] {region}, {feature}, {label}, {surface} , {smth}")

    #print(df_maps.columns)
    col_ID = tsutil.get_IDCol(study, demographics)

    # rename col_ID as ID
    df_demo.rename(columns={col_ID: 'id'}, inplace=True)

    # remove ID, SES cols
    df_maps['ID_SES'] = df_maps[col_ID].astype(str) + '_' + df_maps['SES'].astype(str) # concat ID and SES into single col
    df_maps.set_index('ID_SES', inplace=True)
    df_maps.drop([col_ID, 'SES'], axis=1, inplace=True)

    df_maps.head()

    # add a row to df_maps that randomly samples from a normal distribution centered around the mean of each column
    means = df_maps.mean(axis=0)
    stds = df_maps.std(axis=0)
    random_row = np.random.normal(loc=means, scale=stds)
    random_row_df = pd.DataFrame([random_row], index=['random_sample'], columns=df_maps.columns)
    df_maps = pd.concat([df_maps, random_row_df])
    # add also a row to df_demo with'grp_detailed' = 'random_sample'
    df_demo = pd.concat([df_demo, pd.DataFrame({'grp_detailed': ['random_sample']})], ignore_index=True)
    df_demo.set_index(df_maps.index, inplace=True)

    # plot ridgeplot
    # get range
    min_val = means.mean() - 10 * stds.mean()
    max_val = means.mean() + 10 * stds.mean()

    tsutil.plot_ridgeplot(df_maps, matrix_df=df_demo[['id','SES','grp_detailed']], Range=(min_val, max_val), Xlab = f"[{study}] {region}, {feature}, {label}, {surface}, {smth}")


In [ ]:
# visualize
item= comps[3]
tsutil.printItemMetadata(item, idx=2)
metric = 'm_df_w_TLE_ic_ipsiTo-L_3T'
print(metric)
tsutil.itmToVisual(
                        item=item, 
                        df_name = df_name, 
                        metric = metric, 
                        ipsiTo = ipsiTo,
                        max_val = 2
                        )